In [156]:
import pandas as pd

def get_party_positions(path_party_positions):
    return pd.read_csv(path_party_positions)

def get_parties(party_positions):
    return list(party_positions.columns[2:len(party_positions.keys())])

def get_constituency_positions(path_constituency_results, path_party_positions):
    """
    Liest Wahlkreisdaten aus einer CSV-Datei und erstellt Wahlkreis-Objekte.
    
    :param datei_pfad: Pfad zur CSV-Datei
    :return: Liste von Wahlkreis-Objekten
    """
    # CSV-Datei einlesen
    constituency_results = pd.read_csv(path_constituency_results)
    wahlomat_results = pd.read_csv(path_party_positions)
    
    # Parteien aus den Spalten extrahieren (alle ab der 4. Spalte)
    parties = list(wahlomat_results.columns[2:len(wahlomat_results.keys())])
    #print(parties)
    constituency_positions = {}
    
    for _, row in constituency_results.iterrows():
        id = row['id']
        name = row['wahlkreisname']
        constituency_positions[id] = {'name': name, 'positions': {}}
        
        for question in wahlomat_results['id']:
            constituency_positions[id]['positions'][question] = 0
            
            for party in parties:
                votes = row[party]
                position = wahlomat_results[party][question-1]
                
                constituency_positions[id]['positions'][question] += votes*position
            constituency_positions[id]['positions'][question] /= row['gesamt']
            
    return constituency_positions

def get_party_matchings(constituency_positions, party_positions):
    party_matchings = {}
    
    parties = get_parties(party_positions)
    for c in constituency_positions:
        constituency = constituency_positions[c]
        constituency_matchings = {}
        for p in parties:
            constituency_matchings[p] = 0
            for q in constituency['positions']:
                constituency_matchings[p] += constituency['positions'][q]*party_positions[p][q-1]
        party_matchings[c] = constituency_matchings
    
    return party_matchings

def get_similarity(constituency_positions, constituency_a, constituency_b):
    s = 76
    for q in constituency_positions[constituency_a]['positions']:
        pos_a = constituency_positions[constituency_a]['positions'][q]
        pos_b = constituency_positions[constituency_b]['positions'][q]
        s -= abs(pos_a - pos_b)
    return s / 76
    
def get_constituency_similarities(constituency_positions, constituency):
    constituency_similarities = {}
    
    for c in constituency_positions:
        constituency_similarities[c] = get_similarity(constituency_positions, constituency, c)
        
    return constituency_similarities

In [61]:
wahlkreise = 'bundestagswahl_2021.csv'
wahlomat = 'wahlomat.csv'

constituency_positions = get_constituency_positions(wahlkreise, wahlomat)
party_positions = get_party_positions(wahlomat)
parties = get_parties(party_positions)

In [53]:
party_positions

,id,Thema,CDU,SPD,AfD,FDP,DIE LINKE,BÜNDNIS 90/DIE GRÜNEN,FREIE WÄHLER,PARTEI,...,III. Weg,Bündnis21,LIEBE,LKR,PdF,LfK,SSW,Todenhöfer,Unabhängige,Volt
0,1,Tempolimit auf Autobahnen,-1,1,-1,-1,1,1,-1,1,...,0,-1,1,-1,-1,1,1,-1,-1,1
1,2,Erhöhung der Verteidigungsausgaben,1,1,1,1,-1,-1,1,-1,...,1,-1,1,1,1,-1,0,-1,0,-1
2,3,Wählen ab 16,-1,1,-1,1,1,1,-1,1,...,-1,-1,1,-1,0,1,1,0,0,1
3,4,Windenergie,-1,-1,1,0,-1,-1,-1,-1,...,1,1,1,1,-1,0,-1,-1,1,-1
4,5,Begrenzung für Mieterhöhungen,0,1,-1,-1,1,1,-1,1,...,1,-1,1,-1,0,0,1,-1,1,1
5,6,Patentschutz für Impfstoffe,1,1,1,1,-1,-1,1,-1,...,-1,-1,1,1,-1,-1,0,-1,0,-1
6,7,Ausstieg aus der Kohleverstromung,-1,1,-1,0,1,1,-1,1,...,-1,-1,-1,-1,0,1,1,-1,0,1
7,8,Gesetzliche Rentenversicherung,-1,1,1,-1,1,1,-1,1,...,1,1,1,-1,1,1,1,1,1,1
8,9,Abschaffung des Familiennachzugs,-1,-1,1,0,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,0,-1
9,10,Steuer auf digitale Dienstleistungen,0,1,1,-1,1,1,1,1,...,1,1,1,-1,0,0,1,1,1,1


In [102]:
party_matchings = get_party_matchings(constituency_positions, party_positions)

In [119]:
c = 87
print(constituency_positions[c]['positions'])
# party_matchings[c]

{1: 0.20860344740941755, 2: 0.2007638574802754, 3: 0.43023698247578845, 4: -0.8005140243946214, 5: 0.4534255131270057, 6: 0.20350627813314381, 7: 0.3244384139906528, 8: 0.3167926600762422, 9: -0.8033784899456541, 10: 0.5525546868830451, 11: -0.8996360190390041, 12: 0.2400407773542102, 13: 0.014882298464388016, 14: 0.34508553911538986, 15: 0.5390220614101211, 16: 0.18166742047339063, 17: -0.1493829553531046, 18: -0.10528885155750828, 19: -0.1947764783585679, 20: 0.44107026196578436, 21: 0.9887359738106006, 22: 0.18898293525159196, 23: 0.7444092667973266, 24: 0.4532891100055279, 25: -0.9104764776406568, 26: 0.4100277831621115, 27: 0.25465744868729945, 28: 0.20559539962525036, 29: -0.4987759614625286, 30: 0.5634741157129217, 31: 0.4688534240773047, 32: 0.6702705807183419, 33: 0.04839439167797377, 34: 0.5444925444925445, 35: 0.2513263408785797, 36: 0.45879548864623493, 37: 0.2458558577961563, 38: -0.28731522761373507}


In [151]:
a = 50
b = 41

print(constituency_positions[a]['name'])
print(constituency_positions[b]['name'])
get_similarity(constituency_positions, a, b)

Braunschweig
Stadt Hannover I


0.9967251302171372

In [191]:
constituency_similarities = {}

for c in constituency_positions:
    s = get_constituency_similarities(constituency_positions, c)
    constituency_similarities[c] = s

[83,
 75,
 18,
 19,
 20,
 42,
 5,
 281,
 95,
 86,
 82,
 94,
 129,
 54,
 76,
 153,
 81,
 84,
 87,
 27,
 183,
 21,
 50,
 61,
 41,
 271,
 80,
 96,
 93,
 142,
 182,
 258,
 22,
 11,
 23,
 186,
 140,
 168,
 39,
 55,
 14,
 205,
 132,
 274,
 37,
 220,
 53,
 1,
 219,
 24,
 107,
 193,
 79,
 244,
 143,
 217,
 152,
 115,
 101,
 275,
 85,
 120,
 141,
 102,
 179,
 106,
 290,
 139,
 218,
 259,
 48,
 144,
 160,
 47,
 117,
 7,
 72,
 4,
 26,
 116,
 171,
 119,
 60,
 6,
 12,
 34,
 159,
 296,
 203,
 78,
 2,
 28,
 185,
 8,
 191,
 118,
 13,
 59,
 69,
 52,
 128,
 46,
 114,
 121,
 173,
 184,
 63,
 49,
 40,
 123,
 287,
 10,
 35,
 282,
 57,
 45,
 100,
 103,
 56,
 9,
 15,
 36,
 33,
 113,
 43,
 145,
 122,
 30,
 58,
 29,
 125,
 167,
 62,
 51,
 38,
 162,
 169,
 138,
 133,
 104,
 98,
 135,
 134,
 177,
 199,
 97,
 242,
 252,
 261,
 170,
 3,
 251,
 283,
 105,
 180,
 17,
 187,
 265,
 109,
 44,
 127,
 110,
 111,
 91,
 77,
 209,
 181,
 299,
 88,
 131,
 130,
 297,
 298,
 206,
 112,
 148,
 176,
 288,
 25,
 31,
 64,
 211,
 1

In [208]:
constituencies = [(c, constituency_positions[c]['name'], constituency_similarities[5][c]) for c in range(1,300)]
sorted_constituencies = sorted(constituencies, key= lambda c: c[2])
sorted_constituencies

[(164, 'Erzgebirgskreis I', 0.7556278365350044),
 (156, 'Bautzen I', 0.7582224474096169),
 (157, 'Görlitz', 0.7591105710988039),
 (158, 'Sächsische Schweiz-Osterzgebirge', 0.7603419472321284),
 (227, 'Deggendorf', 0.764835137798943),
 (231, 'Straubing', 0.7675233946611297),
 (230, 'Rottal-Inn', 0.769502503052146),
 (234, 'Schwandorf', 0.7720708318857015),
 (161, 'Mittelsachsen', 0.7743031099570434),
 (155, 'Meißen', 0.7755549124970349),
 (151, 'Nordsachsen', 0.7849184092865465),
 (163, 'Chemnitzer Umland – Erzgebirgskreis II', 0.7859750301570084),
 (212, 'Altötting', 0.7924785695726443),
 (254, 'Donau-Ries', 0.7937842346964428),
 (73, 'Burgenland – Saalekreis', 0.7941097472954854),
 (235, 'Weiden', 0.7952277802247051),
 (229, 'Passau', 0.7975028695917711),
 (74, 'Mansfeld', 0.7991751868820972),
 (248, 'Bad Kissingen', 0.799705756350184),
 (240, 'Kulmbach', 0.8000297537127009),
 (228, 'Landshut', 0.8021134577631915),
 (154, 'Leipzig-Land', 0.8022537981405913),
 (165, 'Zwickau', 0.803263

In [206]:
print(constituency_similarities[95])

{1: 0.9491036184974602, 2: 0.9148599110523676, 3: 0.8951351087225738, 4: 0.9221355474734482, 5: 0.9876746485798172, 6: 0.9182872797738552, 7: 0.9225380361541881, 8: 0.9146327058655807, 9: 0.9036568706384517, 10: 0.9079267506747648, 11: 0.961457877936309, 12: 0.8896176769911162, 13: 0.8781186857815461, 14: 0.936237327983594, 15: 0.8750367637505989, 16: 0.8301326334880471, 17: 0.846772228166521, 18: 0.9678565791014394, 19: 0.972686369526968, 20: 0.9813122397995303, 21: 0.9710718088454431, 22: 0.9569739894631688, 23: 0.953042704501316, 24: 0.9292424670175725, 25: 0.8721168006764838, 26: 0.9106038132938906, 27: 0.9744906694297315, 28: 0.9096287004452046, 29: 0.8927259970629212, 30: 0.8979614493174137, 31: 0.8709675532937683, 32: 0.8322373185817878, 33: 0.8998851958947303, 34: 0.9131041985496356, 35: 0.8991889031665514, 36: 0.9044351642899513, 37: 0.952234339298704, 38: 0.8926244309969864, 39: 0.9556910388529637, 40: 0.9010069662196486, 41: 0.9693992958442971, 42: 0.987366399334267, 43: 0.9

In [207]:
import json

# Dateiname der JSON-Datei
datei_pfad = "constituency_similarities.json"

# Speichern des 2D-Dictionaries
with open(datei_pfad, "w", encoding="utf-8") as file:
    json.dump(constituency_similarities, file, indent=4, ensure_ascii=False)

print(f"Die Datei wurde als {datei_pfad} gespeichert.")

Die Datei wurde als constituency_similarities.json gespeichert.
